In [111]:
import os
import time
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import copy
import logging

In [112]:
input_path_aspect = '../raw_data/restaurants_aspect.csv'
input_path_sentiment = '../raw_data/restaurants_sentiment.csv'

In [113]:
aspect = pd.read_csv(input_path_aspect)
# sentiment = pd.read_csv(input_path_sentiment)

In [114]:
aspect.shape

(37589, 75)

In [115]:
dataTrain = aspect.iloc[:20000,:-1]
dataTest = aspect.iloc[20000:,:-1]
yTrain_o = aspect.iloc[:20000,-1:]
yTest_o = aspect.iloc[20000:,-1:]
dataTrain.shape, dataTest.shape, yTrain_o.shape, yTest_o.shape

((20000, 74), (17589, 74), (20000, 1), (17589, 1))

In [116]:
dataTrain = np.array(dataTrain)
dataTest = np.array(dataTest)
yTrain_o = np.array(yTrain_o)
yTest_o = np.array(yTest_o)

In [117]:
dataTrain[0], yTrain_o[0]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), array(['NO'], dtype=object))

In [118]:
yTrain = []
for i in yTrain_o:
    if i == 'NO':
        yTrain.append([0,1])
    else:
        yTrain.append([1,0])
yTrain = np.array(yTrain)

yTest = []
for i in yTest_o:
    if i == 'NO':
        yTest.append([0,1])
    else:
        yTest.append([1,0])
yTest = np.array(yTest)

In [119]:
yTrain.shape,yTest.shape

((20000, 2), (17589, 2))

In [120]:
yTrain[:10]

array([[0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1]])

In [121]:
numTrain = dataTrain.shape[0]
numTest = dataTest.shape[0]
numTerms = dataTrain.shape[1]
numClasses = yTrain.shape[1]
numTrain,numTest,numTerms,numClasses

(20000, 17589, 74, 2)

# preprocess

In [122]:
'''
fbridge: bridge object对于各个class的score; 初始化为0（对应fTerms）[numTerms * numClasses]
yTrain: 就是yDoc的Train数据部分 [numTrain * numclasses]
'''
fbridge = np.zeros((numTerms,numClasses))
fbridge.shape

(74, 2)

In [123]:
def normalize(Data):
    newData = []
    for i in Data:
        newData.append(i/np.sum(i))
    return np.array(newData)

In [129]:
def get_logger(logpath, package_files=[],
			   displaying=True, saving=True, debug=False):
	logger = logging.getLogger()
	if debug:
		level = logging.DEBUG
	else:
		level = logging.INFO
	logger.setLevel(level)
	if saving:
		info_file_handler = logging.FileHandler(logpath, mode='w')
		info_file_handler.setLevel(level)
		logger.addHandler(info_file_handler)
	if displaying:
		console_handler = logging.StreamHandler()
		console_handler.setLevel(level)
		logger.addHandler(console_handler)

	for f in package_files:
		logger.info(f)
		with open(f, 'r') as package_f:
			logger.info(package_f.read())

	return logger


'''
log_path = "logs/.log"
if not os.path.exists("logs/"):
    utils.makedirs("logs/")
logger = get_logger(logpath=log_path)
logger.info(str(time.asctime( time.localtime(time.time()) )))
'''

'\nlog_path = "logs/.log"\nif not os.path.exists("logs/"):\n    utils.makedirs("logs/")\nlogger = get_logger(logpath=log_path)\nlogger.info(str(time.asctime( time.localtime(time.time()) )))\n'

In [130]:
dataTrain = normalize(dataTrain)
dataTest = normalize(dataTest)

In [131]:
dataTrain[:2]

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

# TCHN

In [132]:
class TCHN(object):
    
    def __init__(self,maxNumberGlobalIterations=100,maxNumberLocalIterations=100,errorCorrectionRate=0.1,minError=0.01):
        self.maxNumberGlobalIterations = maxNumberGlobalIterations
        self.maxNumberLocalIterations = maxNumberLocalIterations
        self.errorCorrectionRate = errorCorrectionRate
        self.minError = minError
        self.numIterations = 0
        
    
    def classifyInstance(self,weightMatrix):
        '''
        利用与sample相连的bridge objects的classvalue，更新该sample的class
        input: [numTrain * numTerms] 一个sample的bridge objects
        return: [numTrain * numclasses]
        '''
        
        # equation 33
        classes = np.dot(weightMatrix,fbridge)
        
        for i in classes:
            # 如果classes[i]的值小于0，则集体偏移
            if min(i) < 0:
                i += abs(min(i))
            i = i/np.sum(i)
        
        return classes
        
        
    def classifyInstanceReal(self,weightMatrix):
        '''
        利用与sample相连的bridge objects的classvalue，更新该sample的class
        input: [numTrain * numTerms] 一个sample的bridge objects
        return: [numTrain * numclasses]
        '''
        
        # equation 33
        classes = np.dot(weightMatrix,fbridge)
        
        return classes 
        
        
    def train(self,fbridge):
        log_path = "logs/test.log"
        if not os.path.exists("logs/"):
            utils.makedirs("logs/")
        logger = get_logger(logpath=log_path)

        exit = False
        numIt = 0
        numIterationsTotal = 0
        while exit == False:
            print(numIterationsTotal)
            logger.info(str(numIterationsTotal))
            logger.info(str(time.asctime( time.localtime(time.time()) )))
            exit2 = False
            numIterationsInternas = 0
            
#             print('First')
            '''First Step: Optimizing class information of terms considering labeled documents'''
            while exit2 == False:
#                 print(numIt)
                
                estimatedClasses = self.classifyInstance(dataTrain)
                error = yTrain - estimatedClasses
                fbridge += self.errorCorrectionRate * np.dot(dataTrain.T,error)
                
                numIt += 1
                numIterationsInternas += 1
                
                # error: [numTrain,numClasses]
                error = pow(error,2)/2
                meanError = np.sum(error)/numTrain
                
                if self.maxNumberLocalIterations == numIterationsInternas or meanError < self.minError:
                    exit2 = True
            
#             print('Second')
            '''Second Step: Propagating class information from terms to unlabeled documents'''
             #基于term，将labeled的document的label传递给unlabeled的document
            fTest = self.classifyInstance(dataTest)
            acmDif = np.sum(abs(yTest - fTest))
            
            
#             print('Third')
            '''Third Step: Optimizing class information of terms considering the class information assigned to unlabeled documents'''
            exit2 = False
            numIterationsInternas = 0
            while exit2 == False:
#                 print(numIt)
                
                estimatedClasses = self.classifyInstanceReal(dataTest)
                error = fTest - estimatedClasses
                fbridge += self.errorCorrectionRate * np.dot(dataTest.T,error)
                
                numIt += 1
                numIterationsInternas += 1
                
                # error: [numTrain,numClasses]
                error = pow(error,2)/2
                meanError = np.sum(error)/numTrain
                
                if self.maxNumberLocalIterations == numIterationsInternas or meanError < self.minError:
                    exit2 = True
                
            numIterationsTotal += 1
            numIt += 1
            if acmDif == 0 or self.maxNumberGlobalIterations == numIterationsTotal:
                exit = True
    
        self.numIterations = numIt
        
        '''Assingning labels to unlabaled documents'''
        fTest = self.classifyInstance(dataTest)
        
        logger.info(str(fTest))
        
        return fTest

In [ ]:
model = TCHN(maxNumberGlobalIterations=100,maxNumberLocalIterations=100)
fTest = model.train(fbridge)

0
0
Mon Jul 27 00:13:01 2020
Mon Jul 27 00:13:01 2020


0


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
1
1
Mon Jul 27 00:13:33 2020
Mon Jul 27 00:13:33 2020


1


2
2
Mon Jul 27 00:14:05 2020
Mon Jul 27 00:14:05 2020


...

5


6
6
Mon Jul 27 00:16:12 2020
Mon Jul 27 00:16:12 2020


6


7
7
Mon Jul 27 00:16:44 2020
Mon Jul 27 00:16:44 2020


7


In [ ]:
a = np.array([[1,2],[3,4]])
b = pow(a,2)/2
b